In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from functions import *
from config import *
import os
from datetime import date
import json
from datetime import datetime

In [ ]:
# get configuration files
outp="/Users/w.zijian/AABC_Behavioral_QC-main/tmp/"
config = LoadSettings("/Users/w.zijian/AABC_Behavioral_QC-main/config.yml")
secret=pd.read_csv(config['config_files']['secrets'])
intradb=pd.read_csv(config['config_files']['PCP'])
user = intradb.user[0]
passw = intradb.auth[0]
HOST = "hcpi-shadow22.nrg.wustl.edu"
projects=["AABC_UMN_ITK","AABC_UCLA_ITK","AABC_MGH_ITK","AABC_STG","AABC_WU_ITK"]#"AABC_WU_ITK",

In [ ]:
# load intraDB dataset
Alldf=pd.DataFrame()
for PROJECT in projects:
    curlcmd = f"""curl -s -k -u {user}:{passw} "https://intradb.humanconnectome.org/data/subjects?xsiType=xnat:subjectData&format=csv&columns=ID,gender,dob,handedness,label,yob&project={PROJECT}" -X GET > {outp}{PROJECT}.csv"""
    os.system(curlcmd)
    projdf=pd.read_csv(outp+PROJECT+".csv")
    projdf['PROJECT']=PROJECT
    Alldf=pd.concat([Alldf,projdf],axis=0)
Alldf

In [ ]:
# Step 1: Merge 'dob' and 'yob' into a single 'yob' column
Alldf['yob'] = Alldf.apply(lambda row: pd.to_datetime(row['dob']).year if pd.notnull(row['dob']) else row['yob'], axis=1)
# Convert 'yob' to integer to remove the decimal and zero after the year
Alldf['yob'] = Alldf['yob'].fillna(0).astype(int)


# Step 2: Drop the 'dob', 'ID', and 'URI' columns
Alldf = Alldf.drop(columns=['dob', 'ID', 'URI'])

# Replace zeros back to NaN to maintain original missing information
#Alldf['yob'] = Alldf['yob'].replace(0, np.nan)

Alldf

In [ ]:
Alldf = Alldf.rename(columns={'label': 'subject'})

In [ ]:
# Exclude the 'label' column
plot_data = Alldf.drop(columns=['subject'])

# For categorical data, we can use bar plots or count plots
for column in plot_data.select_dtypes(include=['object', 'category']).columns:
    plot_data[column].value_counts().plot(kind='bar')
    plt.title(column)
    plt.show()

In [ ]:
aabcarms = redjson(tok=secret.loc[secret.source=='aabcarms','api_key'].reset_index().drop(columns='index').api_key[0])
aabc=getframe(struct=aabcarms,api_url=config['Redcap']['api_url10'])

In [ ]:
def idvisits(aabcarmsdf,keepsies):
    #idvisit=aabcarmsdf[keepsies].copy()
    idvisit = aabcarmsdf[keepsies + ['redcap_event_name', 'study_id']].copy()
    
    #registers=idvisit.loc[idvisit.redcap_event_name.str.contains('register')][['subject_id','study_id','site']]
    registers = idvisit.loc[idvisit['redcap_event_name'].str.contains('register'), ['subject_id', 'study_id', 'site', 'sex']]
    
    idvisit = pd.merge(registers, idvisit.drop(columns=['site']), on='study_id', how='right')
    
    idvisit=idvisit.rename(columns={'subject_id_x':'subject','subject_id_y':'subject_id','sex_x':'sex'})
    
    idvisit['redcap_event']=idvisit.replace({'redcap_event_name':
                                           config['Redcap']['datasources']['aabcarms']['AABCeventmap']})['redcap_event_name']
    
    idvisit = idvisit.loc[~(idvisit.subject.astype(str).str.upper().str.contains('TEST'))]
    #idvisit = idvisit.loc[~idvisit['subject'].astype(str).str.upper().contains('TEST')]
    
    return idvisit

In [ ]:
keepsies=['site','subject_id','sex','dob','event_date']
records = idvisits(aabc,keepsies)
records_evt = records.loc[(records['redcap_event'] == "V1") | (records['redcap_event'] == "V2") | (records['redcap_event'] == "V3") ]
#records = records.loc[records['lab_results_complete'] == "2"]
#records = records.loc[records['bld_drawresults'] == "1"]
records_evt = records_evt.drop(columns=['sex_y','subject_id'])

In [ ]:
records_evt

In [ ]:
# Filter rows where 'dob' is not null or NaN
records_dob = records[pd.notnull(records['dob'])]

# To ensure that 'dob' contains valid dates, you may need to convert the column to datetime
# This step will also filter out any non-date entries
records_dob['dob'] = pd.to_datetime(records_dob['dob'], errors='coerce')
records_dob = records_dob[pd.notnull(records_dob['dob'])]
records_dob = records_dob.drop(columns=['sex_y','subject_id'])

In [ ]:
records_dob = records_dob.reset_index(drop=True)

In [ ]:
records_dob

In [ ]:
# Convert 'dob' to datetime and extract the year
records_dob['dob'] = pd.to_datetime(records_dob['dob'], errors='coerce')
records_dob['yob'] = records_dob['dob'].dt.year

# Keep only the 'subject' and 'year' columns
records_dob = records_dob[['subject', 'yob']].dropna()

records_dob

In [ ]:
# Merge the two dataframes on the 'subject' column
comparison_df = pd.merge(Alldf, records, on='subject', how='inner', suffixes=('_IntraDB', '_RedCap'))

# Find rows where 'yob' values are different between the two dataframes
differences = comparison_df[comparison_df['yob_IntraDB'] != comparison_df['yob_RedCap']]

print("Differences found in yob for the same subjects:")
print(differences)

In [ ]:
differences = differences[differences["yob_IntraDB"] != 0]
differences

In [ ]:
# Specify the columns you want to select
columns_to_select = ['redcap_event', 'study_id', 'site', 'event_date']

Issues_yob = pd.merge(differences, records_evt[['subject'] + columns_to_select], on='subject', how='left')

# Display the merged dataframe
Issues_yob

In [ ]:
# change dataframe to jira ticket form
Issues_yob['issueCode'] = 'AE7001'
Issues_yob['datatype'] = 'REDCap'
Issues_yob['code'] = 'RED'
Issues_yob['reason'] = 'Discrepancy in year of birth between REDCap and IntraDB.  Please correct IntraDB'
Issues_yob['event_date'] = pd.to_datetime(Issues_yob['event_date'])
Issues_yob['issue_age'] = (datetime.now() - Issues_yob['event_date']).dt.days
Issues_yob['event_date'] = Issues_yob['event_date'].astype(str)
Issues_yob['issue_age'] = Issues_yob['issue_age'].astype(str)
Issues_yob['issue_age'] = Issues_yob['issue_age'] + ' days'
Issues_yob = Issues_yob[['subject', 'redcap_event', 'study_id', 'site', 'reason', 'code', 'issueCode', 'event_date', 'issue_age', 'datatype']].dropna()

Issues_yob

In [ ]:
Issues_yob.to_csv('Issues_yob.csv', index=False)